In [ ]:
import tkinter as tk
from tkinter import ttk
import time
import collections
from collections import Counter

# Define functions

def read_second_to_last_line_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            if len(lines) >= 2:
                return lines[-2].strip()
            else:
                return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None


def get_decision(behavior, accelerometer, detection):
    decision_map = {
       ('Safe', 'Slow', 'Safe'): 0,
        ('Safe', 'Slow', 'Normal'): 0,
        ('Safe', 'Slow', 'Ready to Take Over'): 0,
        ('Safe', 'Slow', 'Take Over'): 1,
        ('Safe', 'Normal', 'Safe'): 0,
        ('Safe', 'Normal', 'Normal'): 0,
        ('Safe', 'Normal', 'Ready to Take Over'): 1,
        ('Safe', 'Normal', 'Take Over'): 1,
        ('Safe', 'Aggressive', 'Safe'): 0,
        ('Safe', 'Aggressive', 'Normal'): 0,
        ('Safe', 'Aggressive', 'Ready to Take Over'): 1,
        ('Safe', 'Aggressive', 'Take Over'): 2,
        ('Talking Phone', 'Slow', 'Safe'): 0,
        ('Talking Phone', 'Slow', 'Normal'): 0,
        ('Talking Phone', 'Slow', 'Ready to Take Over'): 1,
        ('Talking Phone', 'Slow', 'Take Over'): 2,
        ('Talking Phone', 'Normal', 'Safe'): 0,
        ('Talking Phone', 'Normal', 'Normal'): 0,
        ('Talking Phone', 'Normal', 'Ready to Take Over'): 1,
        ('Talking Phone', 'Normal', 'Take Over'): 2,
        ('Talking Phone', 'Aggressive', 'Safe'): 1,
        ('Talking Phone', 'Aggressive', 'Normal'): 1,
        ('Talking Phone', 'Aggressive', 'Ready to Take Over'): 2,
        ('Talking Phone', 'Aggressive', 'Take Over'): 2,
        ('Texting Phone', 'Slow', 'Safe'): 0,
        ('Texting Phone', 'Slow', 'Normal'): 0,
        ('Texting Phone', 'Slow', 'Ready to Take Over'): 1,
        ('Texting Phone', 'Slow', 'Take Over'): 2,
        ('Texting Phone', 'Normal', 'Safe'): 0,
        ('Texting Phone', 'Normal', 'Normal'): 0,
        ('Talking Phone', 'Normal', 'Ready to Take Over'): 1,
        ('Talking Phone', 'Normal', 'Take Over'): 2,
        ('Talking Phone', 'Aggressive', 'Safe'): 1,
        ('Talking Phone', 'Aggressive', 'Normal'): 1,
        ('Talking Phone', 'Aggressive', 'Ready to Take Over'): 2,
        ('Talking Phone', 'Aggressive', 'Take Over'): 2,
        ('Turning', 'Slow', 'Safe'): 0,
        ('Turning', 'Slow', 'Normal'): 0,
        ('Turning', 'Slow', 'Ready to Take Over'): 1,
        ('Turning', 'Slow', 'Take Over'): 2,
        ('Turning', 'Normal', 'Safe'): 0,
        ('Turning', 'Normal', 'Normal'): 0,
        ('Turning', 'Normal', 'Ready to Take Over'): 1,
        ('Turning', 'Normal', 'Take Over'): 2,
        ('Turning', 'Aggressive', 'Safe'): 0,
        ('Turning', 'Aggressive', 'Normal'): 1,
        ('Turning', 'Aggressive', 'Ready to Take Over'): 2,
        ('Turning', 'Aggressive', 'Take Over'): 2,
        ('Other', 'Slow', 'Safe'): 0,
        ('Other', 'Slow', 'Normal'): 0,
        ('Other', 'Slow', 'Ready to Take Over'): 1,
        ('Other', 'Slow', 'Take Over'): 2,
        ('Other', 'Normal', 'Safe'): 0,
        ('Other', 'Normal', 'Normal'): 0,
        ('Other', 'Normal', 'Ready to Take Over'): 1,
        ('Other', 'Normal', 'Take Over'): 2,
        ('Other', 'Aggressive', 'Safe'): 0,
        ('Other', 'Aggressive', 'Normal'): 1,
        ('Other', 'Aggressive', 'Ready to Take Over'): 2,
        ('Other', 'Aggressive', 'Take Over'): 2
    }
    decision = decision_map.get((behavior, accelerometer, detection), "Unknown")
    if isinstance(decision, tuple):  # Handle case for turning with aggressive and normal behavior
        return decision[0]  # Return the first decision for normal behavior
    return decision

def update_gui(behavior_label, accelerometer_label, detection_label, decision_label, root, behavior, accelerometer, detection, decision):
    behavior_label.config(text=behavior)
    accelerometer_label.config(text=accelerometer)
    detection_label.config(text=detection)
    if decision == 0:
        decision_label.config(text="Keep watching Do not take over")
        root.configure(background='green')
    elif decision == 1:
        decision_label.config(text="Ready to take over")
        root.configure(background='orange')
    elif decision == 2:
        decision_label.config(text="Take over")
        root.configure(background='red')
        root.after(500, lambda: root.configure(background='white'))
    root.update()

def calculate_avg_last_10_outputs(last_10_outputs):
    if not last_10_outputs:
        return 0
    avg = sum(last_10_outputs) / len(last_10_outputs)
    if avg <= 0.5:
        return 0
    elif avg <= 1.5:
        return 1
    else:
        return 2
    
def calculate_avg_last_10_inputs_by_file(last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3):
    if not last_10_inputs_file1 or not last_10_inputs_file2 or not last_10_inputs_file3:
        return 0
    file1_avg = sum(last_10_inputs_file1) / len(last_10_inputs_file1)
    file2_avg = sum(last_10_inputs_file2) / len(last_10_inputs_file2)
    file3_avg = sum(last_10_inputs_file3) / len(last_10_inputs_file3)
    avg = (file1_avg + file2_avg + file3_avg) / 3
    if avg <= 0.5:
        return 0
    elif avg <= 1.5:
        return 1
    else:
        return 2
    
def calculate_avg_last_10_inputs(last_10_inputs):
    # Define a dictionary to map strings to numbers
    behavior_map = {
        'Safe': 0,
        'Talking on Phone': 1,
        'Texting on Phone': 2,
        'Turning': 3,
        'Other': 4
        # Add more mappings as needed
    }
    accelerometer_map = {
        'Slow': 0,
        'Normal': 1,
        'Aggressive': 2
        # Add more mappings as needed
    }
    detection_map = {
        'Safe': 0,
        'Normal': 1,
        'Ready to Take Over': 2,
        'Take Over': 3
        # Add more mappings as needed
    }

    if not last_10_inputs:
        return 0

    # Replace strings with numbers
    numeric_inputs = []
    for input_data in last_10_inputs:
        behavior, accelerometer, detection = input_data
        numeric_behavior = behavior_map.get(behavior, -1)
        numeric_accelerometer = accelerometer_map.get(accelerometer, -1)
        numeric_detection = detection_map.get(detection, -1)
        numeric_inputs.append((numeric_behavior, numeric_accelerometer, numeric_detection))

    # Calculate average
    avg_behavior = sum(input_tuple[0] for input_tuple in numeric_inputs) / len(numeric_inputs)
    avg_accelerometer = sum(input_tuple[1] for input_tuple in numeric_inputs) / len(numeric_inputs)
    avg_detection = sum(input_tuple[2] for input_tuple in numeric_inputs) / len(numeric_inputs)

    # Determine the overall average based on the averages of individual components
    overall_avg = (avg_behavior + avg_accelerometer + avg_detection) / 3

    # Determine the result based on overall_avg
    if overall_avg <= 0.5:
        return 0
    elif overall_avg <= 1.5:
        return 1
    else:
        return 2

def calculate_weighted_avg_last_10_outputs(last_10_outputs):
    if not last_10_outputs:
        return 0
    weights = collections.Counter(last_10_outputs)
    weighted_sum = sum(val * weight for val, weight in weights.items())
    total_weights = sum(weights.values())
    weighted_avg = weighted_sum / total_weights
    if weighted_avg <= 0.5:
        return 0
    elif weighted_avg <= 1.5:
        return 1
    else:
        return 2

def calculate_weighted_avg_last_10_inputs_by_file(last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3):
    # Define a dictionary to map strings to numbers
    behavior_map = {
        'Safe': 0,
        'Talking on Phone': 1,
        'Texting on Phone': 2,
        'Turning': 3,
        'Other': 4
        # Add more mappings as needed
    }
    accelerometer_map = {
        'Slow': 0,
        'Normal': 1,
        'Aggressive': 2
        # Add more mappings as needed
    }
    detection_map = {
        'Safe': 0,
        'Normal': 1,
        'Ready to Take Over': 2,
        'Take Over': 3
        # Add more mappings as needed
    }

    # Replace strings with numbers
    numeric_inputs1 = []
    for input_data in last_10_inputs_file1:
        numeric_behavior = behavior_map.get(input_data, -1)
        numeric_inputs1.append((numeric_behavior,))

    # Replace strings with numbers
    numeric_inputs2 = []
    for input_data in last_10_inputs_file2:
        numeric_accelerometer = accelerometer_map.get(input_data, -1)
        numeric_inputs2.append((numeric_accelerometer,))

    # Replace strings with numbers
    numeric_inputs3 = []
    for input_data in last_10_inputs_file3:
        numeric_detection = detection_map.get(input_data, -1)
        numeric_inputs3.append((numeric_detection,))

    if not last_10_inputs_file1 or not last_10_inputs_file2 or not last_10_inputs_file3:
        return 0
    weights_file1 = collections.Counter(input_tuple[0] for input_tuple in numeric_inputs1)
    weights_file2 = collections.Counter(input_tuple[0] for input_tuple in numeric_inputs2)
    weights_file3 = collections.Counter(input_tuple[0] for input_tuple in numeric_inputs3)
    weighted_sum_file1 = sum(val * weight for val, weight in weights_file1.items())
    weighted_sum_file2 = sum(val * weight for val, weight in weights_file2.items())
    weighted_sum_file3 = sum(val * weight for val, weight in weights_file3.items())
    total_weights_file1 = sum(weights_file1.values())
    total_weights_file2 = sum(weights_file2.values())
    total_weights_file3 = sum(weights_file3.values())
    weighted_avg_file1 = weighted_sum_file1 / total_weights_file1
    weighted_avg_file2 = weighted_sum_file2 / total_weights_file2
    weighted_avg_file3 = weighted_sum_file3 / total_weights_file3
    weighted_avg = (weighted_avg_file1 + weighted_avg_file2 + weighted_avg_file3) / 3
    if weighted_avg <= 0.5:
        return 0
    elif weighted_avg <= 1.5:
        return 1
    else:
        return 2

def calculate_weighted_avg_last_10_inputs(last_10_inputs):
    # Define a dictionary to map strings to numbers
    behavior_map = {
        'Safe': 0,
        'Talking on Phone': 1,
        'Texting on Phone': 2,
        'Turning': 3,
        'Other': 4
        # Add more mappings as needed
    }
    accelerometer_map = {
        'Slow': 0,
        'Normal': 1,
        'Aggressive': 2
        # Add more mappings as needed
    }
    detection_map = {
        'Safe': 0,
        'Normal': 1,
        'Ready to Take Over': 2,
        'Take Over': 3
        # Add more mappings as needed
    }

    if not last_10_inputs:
        return 0

    # Replace strings with numbers
    numeric_inputs = []
    for input_data in last_10_inputs:
        behavior, accelerometer, detection = input_data
        numeric_behavior = behavior_map.get(behavior, -1)
        numeric_accelerometer = accelerometer_map.get(accelerometer, -1)
        numeric_detection = detection_map.get(detection, -1)
        numeric_inputs.append((numeric_behavior, numeric_accelerometer, numeric_detection))

    # Calculate weighted average
    weights_behavior = collections.Counter(input_tuple[0] for input_tuple in numeric_inputs)
    weights_accelerometer = collections.Counter(input_tuple[1] for input_tuple in numeric_inputs)
    weights_detection = collections.Counter(input_tuple[2] for input_tuple in numeric_inputs)
    weighted_sum_behavior = sum(val * weight for val, weight in weights_behavior.items())
    weighted_sum_accelerometer = sum(val * weight for val, weight in weights_accelerometer.items())
    weighted_sum_detection = sum(val * weight for val, weight in weights_detection.items())
    total_weights_behavior = sum(weights_behavior.values())
    total_weights_accelerometer = sum(weights_accelerometer.values())
    total_weights_detection = sum(weights_detection.values())
    weighted_avg_behavior = weighted_sum_behavior / total_weights_behavior
    weighted_avg_accelerometer = weighted_sum_accelerometer / total_weights_accelerometer
    weighted_avg_detection = weighted_sum_detection / total_weights_detection

    # Determine the overall weighted average based on the weighted averages of individual components
    overall_weighted_avg = (weighted_avg_behavior + weighted_avg_accelerometer + weighted_avg_detection) / 3

    # Determine the result based on overall_weighted_avg
    if overall_weighted_avg <= 0.5:
        return 0
    elif overall_weighted_avg <= 1.5:
        return 1
    else:
        return 2

def update_after_decision(behavior, accelerometer, detection, decision, last_10_outputs, last_10_inputs, last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3):
    last_10_outputs.append(decision)
    last_10_inputs.append((behavior, accelerometer, detection))  # Append a tuple
    last_10_inputs_file1.append(behavior)
    last_10_inputs_file2.append(accelerometer) 
    last_10_inputs_file3.append(detection)

    # Calculate weighted average decision
    w_output_avg = calculate_weighted_avg_last_10_outputs(last_10_outputs)
    w_input_avg = calculate_weighted_avg_last_10_inputs(last_10_inputs)
    w_input_file_avg = calculate_weighted_avg_last_10_inputs_by_file(last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3)

    print(f"\nOriginal decision: {decision}")

    print(f"Weighted Output average decision: {w_output_avg}")
    print(f"Weighted Input average decision: {w_input_avg}")
    print(f"Weighted Input file average decision: {w_input_file_avg}")

    # Example usage:
    original_decision = decision  
    weighted_output_average_decision = w_output_avg  
    weight_input_average_decision = w_input_avg  
    weighted_input_file_average_decision = w_input_file_avg  

    decisions = [original_decision, weighted_output_average_decision, weight_input_average_decision, weighted_input_file_average_decision]
    weights = [2, 1, 1, 1]  # Define your weights here

    weighted_average = calculate_weighted_average(decisions, weights)
    #print('Weighted average:', weighted_average)
    return weighted_average

def calculate_weighted_average(decisions, weights):
    # Calculate the frequency of each decision
    frequency = Counter(decisions)

    # Calculate the weighted average of decisions based on their frequency
    total_weight = 0
    for decision, weight in zip(frequency, weights):
        total_weight += decision * frequency[decision] * weight

    weighted_average = total_weight / sum(frequency.values())

    # Determine the result based on overall_weighted_avg
    if weighted_average < 0.75:
        final_avg = 0
    elif weighted_average < 1.5:
        final_avg = 1
    else:
        final_avg = 2

    return final_avg

def main():
    root = tk.Tk()
    root.title("Driver Assist System")
    root.geometry("600x200")

    behavior_label = ttk.Label(root, text="")
    behavior_label.grid(row=0, column=0, padx=10, pady=10)

    accelerometer_label = ttk.Label(root, text="")
    accelerometer_label.grid(row=0, column=1, padx=10, pady=10)

    detection_label = ttk.Label(root, text="")
    detection_label.grid(row=0, column=2, padx=10, pady=10)

    decision_label = ttk.Label(root, text="")
    decision_label.grid(row=0, column=3, padx=10, pady=10)

    file_paths = [
        "C:\\Users\\14374\\Desktop\\Vatsal\\predictions.txt",
        "C:\\Users\\14374\\Desktop\\Vatsal\\output.txt",
        "C:\\Users\\14374\\Desktop\\Vatsal\\predictions2.txt"
    ]

    last_10_outputs = collections.deque(maxlen=10) 
    last_10_inputs = collections.deque(maxlen=30)  # Increased to accommodate three inputs
    last_10_inputs_file1 = collections.deque(maxlen=10)
    last_10_inputs_file2 = collections.deque(maxlen=10)
    last_10_inputs_file3 = collections.deque(maxlen=10)

    while True:
        data = [read_second_to_last_line_from_file(file_path) for file_path in file_paths]
        if all(data):
            behavior, accelerometer, detection = data
            decision = get_decision(behavior, accelerometer, detection)
            weighted_average_decision = update_after_decision(behavior, accelerometer, detection, decision, last_10_outputs, last_10_inputs, last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3)
            update_gui(behavior_label, accelerometer_label, detection_label, decision_label, root, behavior, accelerometer, detection, weighted_average_decision)
        time.sleep(1 / 30)  # Adjust the delay as needed

    root.mainloop()  # Moved outside the loop

if __name__ == "__main__":
    main()


In [1]:
import tkinter as tk
from tkinter import ttk
import time
import collections
from collections import Counter

# Define functions

def read_second_to_last_line_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            if len(lines) >= 2:
                return lines[-2].strip()
            else:
                return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None


def get_decision(behavior, accelerometer, detection):
    decision_map = {
       ('Safe', 'Slow', 'Safe'): 0,
        ('Safe', 'Slow', 'Normal'): 0,
        ('Safe', 'Slow', 'Ready to Take Over'): 0,
        ('Safe', 'Slow', 'Take Over'): 1,
        ('Safe', 'Normal', 'Safe'): 0,
        ('Safe', 'Normal', 'Normal'): 0,
        ('Safe', 'Normal', 'Ready to Take Over'): 1,
        ('Safe', 'Normal', 'Take Over'): 1,
        ('Safe', 'Aggressive', 'Safe'): 0,
        ('Safe', 'Aggressive', 'Normal'): 0,
        ('Safe', 'Aggressive', 'Ready to Take Over'): 1,
        ('Safe', 'Aggressive', 'Take Over'): 2,
        ('Talking Phone', 'Slow', 'Safe'): 0,
        ('Talking Phone', 'Slow', 'Normal'): 0,
        ('Talking Phone', 'Slow', 'Ready to Take Over'): 1,
        ('Talking Phone', 'Slow', 'Take Over'): 2,
        ('Talking Phone', 'Normal', 'Safe'): 0,
        ('Talking Phone', 'Normal', 'Normal'): 0,
        ('Talking Phone', 'Normal', 'Ready to Take Over'): 1,
        ('Talking Phone', 'Normal', 'Take Over'): 2,
        ('Talking Phone', 'Aggressive', 'Safe'): 1,
        ('Talking Phone', 'Aggressive', 'Normal'): 1,
        ('Talking Phone', 'Aggressive', 'Ready to Take Over'): 2,
        ('Talking Phone', 'Aggressive', 'Take Over'): 2,
        ('Texting Phone', 'Slow', 'Safe'): 0,
        ('Texting Phone', 'Slow', 'Normal'): 0,
        ('Texting Phone', 'Slow', 'Ready to Take Over'): 1,
        ('Texting Phone', 'Slow', 'Take Over'): 2,
        ('Texting Phone', 'Normal', 'Safe'): 0,
        ('Texting Phone', 'Normal', 'Normal'): 0,
        ('Talking Phone', 'Normal', 'Ready to Take Over'): 1,
        ('Talking Phone', 'Normal', 'Take Over'): 2,
        ('Talking Phone', 'Aggressive', 'Safe'): 1,
        ('Talking Phone', 'Aggressive', 'Normal'): 1,
        ('Talking Phone', 'Aggressive', 'Ready to Take Over'): 2,
        ('Talking Phone', 'Aggressive', 'Take Over'): 2,
        ('Turning', 'Slow', 'Safe'): 0,
        ('Turning', 'Slow', 'Normal'): 0,
        ('Turning', 'Slow', 'Ready to Take Over'): 1,
        ('Turning', 'Slow', 'Take Over'): 2,
        ('Turning', 'Normal', 'Safe'): 0,
        ('Turning', 'Normal', 'Normal'): 0,
        ('Turning', 'Normal', 'Ready to Take Over'): 1,
        ('Turning', 'Normal', 'Take Over'): 2,
        ('Turning', 'Aggressive', 'Safe'): 0,
        ('Turning', 'Aggressive', 'Normal'): 1,
        ('Turning', 'Aggressive', 'Ready to Take Over'): 2,
        ('Turning', 'Aggressive', 'Take Over'): 2,
        ('Other', 'Slow', 'Safe'): 0,
        ('Other', 'Slow', 'Normal'): 0,
        ('Other', 'Slow', 'Ready to Take Over'): 1,
        ('Other', 'Slow', 'Take Over'): 2,
        ('Other', 'Normal', 'Safe'): 0,
        ('Other', 'Normal', 'Normal'): 0,
        ('Other', 'Normal', 'Ready to Take Over'): 1,
        ('Other', 'Normal', 'Take Over'): 2,
        ('Other', 'Aggressive', 'Safe'): 0,
        ('Other', 'Aggressive', 'Normal'): 1,
        ('Other', 'Aggressive', 'Ready to Take Over'): 2,
        ('Other', 'Aggressive', 'Take Over'): 2
    }
    decision = decision_map.get((behavior, accelerometer, detection), 0)
    if isinstance(decision, tuple):  # Handle case for turning with aggressive and normal behavior
        return decision[0]  # Return the first decision for normal behavior
    return decision

def update_gui(behavior_label, accelerometer_label, detection_label, decision_label, root, behavior, accelerometer, detection, decision):
    behavior_label.config(text=behavior)
    accelerometer_label.config(text=accelerometer)
    detection_label.config(text=detection)
    if decision == 0:
        decision_label.config(text="Keep watching Do not take over")
        root.configure(background='green')
    elif decision == 1:
        decision_label.config(text="Ready to take over")
        root.configure(background='orange')
    elif decision == 2:
        decision_label.config(text="Take over")
        root.configure(background='red')
        root.after(500, lambda: root.configure(background='white'))
    root.update()

def update_final_gui(behavior_label, accelerometer_label, detection_label, decision_label, root2, behavior, accelerometer, detection, decision):
    behavior_label.config(text=behavior)
    accelerometer_label.config(text=accelerometer)
    detection_label.config(text=detection)
    if decision == 0:
        decision_label.config(text="Keep watching Do not take over")
        root2.configure(background='green')
    elif decision == 1:
        decision_label.config(text="Ready to take over")
        root2.configure(background='orange')
    elif decision == 2:
        decision_label.config(text="Take over")
        root2.configure(background='red')
        root2.after(500, lambda: root2.configure(background='white'))
    root2.update()


def calculate_avg_last_10_outputs(last_10_outputs):
    if not last_10_outputs:
        return 0
    avg = sum(last_10_outputs) / len(last_10_outputs)
    if avg <= 0.5:
        return 0
    elif avg <= 1.5:
        return 1
    else:
        return 2
    
def calculate_avg_last_10_inputs_by_file(last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3):
    if not last_10_inputs_file1 or not last_10_inputs_file2 or not last_10_inputs_file3:
        return 0
    file1_avg = sum(last_10_inputs_file1) / len(last_10_inputs_file1)
    file2_avg = sum(last_10_inputs_file2) / len(last_10_inputs_file2)
    file3_avg = sum(last_10_inputs_file3) / len(last_10_inputs_file3)
    avg = (file1_avg + file2_avg + file3_avg) / 3
    if avg <= 0.5:
        return 0
    elif avg <= 1.5:
        return 1
    else:
        return 2
    
def calculate_avg_last_10_inputs(last_10_inputs):
    # Define a dictionary to map strings to numbers
    behavior_map = {
        'Safe': 0,
        'Talking on Phone': 1,
        'Texting on Phone': 2,
        'Turning': 3,
        'Other': 4
        # Add more mappings as needed
    }
    accelerometer_map = {
        'Slow': 0,
        'Normal': 1,
        'Aggressive': 2
        # Add more mappings as needed
    }
    detection_map = {
        'Safe': 0,
        'Normal': 1,
        'Ready to Take Over': 2,
        'Take Over': 3
        # Add more mappings as needed
    }

    if not last_10_inputs:
        return 0

    # Replace strings with numbers
    numeric_inputs = []
    for input_data in last_10_inputs:
        behavior, accelerometer, detection = input_data
        numeric_behavior = behavior_map.get(behavior, -1)
        numeric_accelerometer = accelerometer_map.get(accelerometer, -1)
        numeric_detection = detection_map.get(detection, -1)
        numeric_inputs.append((numeric_behavior, numeric_accelerometer, numeric_detection))

    # Calculate average
    avg_behavior = sum(input_tuple[0] for input_tuple in numeric_inputs) / len(numeric_inputs)
    avg_accelerometer = sum(input_tuple[1] for input_tuple in numeric_inputs) / len(numeric_inputs)
    avg_detection = sum(input_tuple[2] for input_tuple in numeric_inputs) / len(numeric_inputs)

    # Determine the overall average based on the averages of individual components
    overall_avg = (avg_behavior + avg_accelerometer + avg_detection) / 3

    # Determine the result based on overall_avg
    if overall_avg <= 0.5:
        return 0
    elif overall_avg <= 1.5:
        return 1
    else:
        return 2

def calculate_weighted_avg_last_10_outputs(last_10_outputs):
    if not last_10_outputs:
        return 0
    print(last_10_outputs)
    weights = collections.Counter(last_10_outputs)
    weighted_sum = sum(val * weight for val, weight in weights.items())
    total_weights = sum(weights.values())
    weighted_avg = weighted_sum / total_weights
    if weighted_avg <= 0.5:
        return 0
    elif weighted_avg <= 1.5:
        return 1
    else:
        return 2

def calculate_weighted_avg_last_10_inputs_by_file(last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3):
    # Define a dictionary to map strings to numbers
    behavior_map = {
        'Safe': 0,
        'Talking on Phone': 1,
        'Texting on Phone': 2,
        'Turning': 3,
        'Other': 4
        # Add more mappings as needed
    }
    accelerometer_map = {
        'Slow': 0,
        'Normal': 1,
        'Aggressive': 2
        # Add more mappings as needed
    }
    detection_map = {
        'Safe': 0,
        'Normal': 1,
        'Ready to Take Over': 2,
        'Take Over': 3
        # Add more mappings as needed
    }

    # Replace strings with numbers
    numeric_inputs1 = []
    for input_data in last_10_inputs_file1:
        numeric_behavior = behavior_map.get(input_data, -1)
        numeric_inputs1.append((numeric_behavior,))

    # Replace strings with numbers
    numeric_inputs2 = []
    for input_data in last_10_inputs_file2:
        numeric_accelerometer = accelerometer_map.get(input_data, -1)
        numeric_inputs2.append((numeric_accelerometer,))

    # Replace strings with numbers
    numeric_inputs3 = []
    for input_data in last_10_inputs_file3:
        numeric_detection = detection_map.get(input_data, -1)
        numeric_inputs3.append((numeric_detection,))

    if not last_10_inputs_file1 or not last_10_inputs_file2 or not last_10_inputs_file3:
        return 0
    weights_file1 = collections.Counter(input_tuple[0] for input_tuple in numeric_inputs1)
    weights_file2 = collections.Counter(input_tuple[0] for input_tuple in numeric_inputs2)
    weights_file3 = collections.Counter(input_tuple[0] for input_tuple in numeric_inputs3)
    weighted_sum_file1 = sum(val * weight for val, weight in weights_file1.items())
    weighted_sum_file2 = sum(val * weight for val, weight in weights_file2.items())
    weighted_sum_file3 = sum(val * weight for val, weight in weights_file3.items())
    total_weights_file1 = sum(weights_file1.values())
    total_weights_file2 = sum(weights_file2.values())
    total_weights_file3 = sum(weights_file3.values())
    weighted_avg_file1 = weighted_sum_file1 / total_weights_file1
    weighted_avg_file2 = weighted_sum_file2 / total_weights_file2
    weighted_avg_file3 = weighted_sum_file3 / total_weights_file3
    weighted_avg = (weighted_avg_file1 + weighted_avg_file2 + weighted_avg_file3) / 3
    if weighted_avg <= 0.5:
        return 0
    elif weighted_avg <= 1.5:
        return 1
    else:
        return 2

def calculate_weighted_avg_last_10_inputs(last_10_inputs):
    # Define a dictionary to map strings to numbers
    behavior_map = {
        'Safe': 0,
        'Talking on Phone': 1,
        'Texting on Phone': 2,
        'Turning': 3,
        'Other': 4
        # Add more mappings as needed
    }
    accelerometer_map = {
        'Slow': 0,
        'Normal': 1,
        'Aggressive': 2
        # Add more mappings as needed
    }
    detection_map = {
        'Safe': 0,
        'Normal': 1,
        'Ready to Take Over': 2,
        'Take Over': 3
        # Add more mappings as needed
    }

    if not last_10_inputs:
        return 0

    # Replace strings with numbers
    numeric_inputs = []
    for input_data in last_10_inputs:
        behavior, accelerometer, detection = input_data
        numeric_behavior = behavior_map.get(behavior, -1)
        numeric_accelerometer = accelerometer_map.get(accelerometer, -1)
        numeric_detection = detection_map.get(detection, -1)
        numeric_inputs.append((numeric_behavior, numeric_accelerometer, numeric_detection))

    # Calculate weighted average
    weights_behavior = collections.Counter(input_tuple[0] for input_tuple in numeric_inputs)
    weights_accelerometer = collections.Counter(input_tuple[1] for input_tuple in numeric_inputs)
    weights_detection = collections.Counter(input_tuple[2] for input_tuple in numeric_inputs)
    weighted_sum_behavior = sum(val * weight for val, weight in weights_behavior.items())
    weighted_sum_accelerometer = sum(val * weight for val, weight in weights_accelerometer.items())
    weighted_sum_detection = sum(val * weight for val, weight in weights_detection.items())
    total_weights_behavior = sum(weights_behavior.values())
    total_weights_accelerometer = sum(weights_accelerometer.values())
    total_weights_detection = sum(weights_detection.values())
    weighted_avg_behavior = weighted_sum_behavior / total_weights_behavior
    weighted_avg_accelerometer = weighted_sum_accelerometer / total_weights_accelerometer
    weighted_avg_detection = weighted_sum_detection / total_weights_detection

    # Determine the overall weighted average based on the weighted averages of individual components
    overall_weighted_avg = (weighted_avg_behavior + weighted_avg_accelerometer + weighted_avg_detection) / 3

    # Determine the result based on overall_weighted_avg
    if overall_weighted_avg <= 0.5:
        return 0
    elif overall_weighted_avg <= 1.5:
        return 1
    else:
        return 2

def update_after_decision(behavior, accelerometer, detection, decision, last_10_outputs, last_10_inputs, last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3):
    
    last_10_outputs.append(decision)
    last_10_inputs.append((behavior, accelerometer, detection))  # Append a tuple
    last_10_inputs_file1.append(behavior)
    last_10_inputs_file2.append(accelerometer) 
    last_10_inputs_file3.append(detection)

    # Calculate weighted average decision
    w_output_avg = calculate_weighted_avg_last_10_outputs(last_10_outputs)
    w_input_avg = calculate_weighted_avg_last_10_inputs(last_10_inputs)
    w_input_file_avg = calculate_weighted_avg_last_10_inputs_by_file(last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3)

    print(f"\nOriginal decision: {decision}")

    #print(f"Weighted Output average decision: {w_output_avg}")
    #print(f"Weighted Input average decision: {w_input_avg}")
    #print(f"Weighted Input file average decision: {w_input_file_avg}")

    # Example usage:
    original_decision = decision  
    weighted_output_average_decision = w_output_avg  
    weight_input_average_decision = w_input_avg  
    weighted_input_file_average_decision = w_input_file_avg  

    decisions = [original_decision, weighted_output_average_decision, weight_input_average_decision, weighted_input_file_average_decision]
    weights = [2, 1, 1, 1]  # Define your weights here

    weighted_average = calculate_weighted_average(decisions, weights)
    #print('Weighted average:', weighted_average)
    return weighted_average

def calculate_weighted_average(decisions, weights):
    # Calculate the frequency of each decision
    frequency = Counter(decisions)

    # Calculate the weighted average of decisions based on their frequency
    total_weight = 0
    for decision, weight in zip(frequency, weights):
        total_weight += decision * frequency[decision] * weight

    weighted_average = total_weight / sum(frequency.values())

    # Determine the result based on overall_weighted_avg
    if weighted_average <= 0.75:
        final_avg = 0
    elif weighted_average <= 1.5:
        final_avg = 1
    else:
        final_avg = 2

    return final_avg

def main():
    root = tk.Tk()
    root.title("Driver Assist System")
    root.geometry("600x200")

    # Create a second window for the weighted average
    root2 = tk.Tk()
    root2.title("Weighted Average")
    root2.geometry("600x200")

    behavior_label = ttk.Label(root, text="")
    behavior_label.grid(row=0, column=0, padx=10, pady=10)

    accelerometer_label = ttk.Label(root, text="")
    accelerometer_label.grid(row=0, column=1, padx=10, pady=10)

    detection_label = ttk.Label(root, text="")
    detection_label.grid(row=0, column=2, padx=10, pady=10)

    decision_label = ttk.Label(root, text="")
    decision_label.grid(row=0, column=3, padx=10, pady=10)

    # Add labels for the weighted average in the second window
    weighted_avg_label = ttk.Label(root2, text="")
    weighted_avg_label.grid(row=0, column=0, padx=10, pady=10)

    file_paths = [
        "C:\\Users\\14374\\Desktop\\Vatsal\\predictions.txt",
        "C:\\Users\\14374\\Desktop\\Vatsal\\output.txt",
        "C:\\Users\\14374\\Desktop\\Vatsal\\predictions2.txt"
    ]

    last_10_outputs = collections.deque(maxlen=10) 
    last_10_inputs = collections.deque(maxlen=30)  # Increased to accommodate three inputs
    last_10_inputs_file1 = collections.deque(maxlen=10)
    last_10_inputs_file2 = collections.deque(maxlen=10)
    last_10_inputs_file3 = collections.deque(maxlen=10)

    '''take_over_counter = 0
    take_over_lock = False'''

    take_over_count = 0
    take_over_lock = False
    lock_counter = 0


    while True:
        data = [read_second_to_last_line_from_file(file_path) for file_path in file_paths]
        if all(data):
            behavior, accelerometer, detection = data
            decision = get_decision(behavior, accelerometer, detection)

            weighted_average = update_after_decision(behavior, accelerometer, detection, decision, last_10_outputs, last_10_inputs, last_10_inputs_file1, last_10_inputs_file2, last_10_inputs_file3)
                
            '''if weighted_average == 2:
                take_over_counter += 1
            else:
                take_over_counter = 0

            if take_over_counter >= 10:
                take_over_lock = True

            if take_over_lock:
                if take_over_counter <= 15:  # 15 + 50
                    weighted_average = 2
                else:
                    take_over_lock = False
                    take_over_counter = 0'''
            
            # Check if the output has passed through 2: Take Over for 15 times
            if decision == 2:
                take_over_count += 1
                if take_over_count >= 20:
                    take_over_lock = True
            else:
                take_over_count = 0

            # If take_over_lock is True, keep the weighted average at 2: Take Over for the next 50 outputs
            if take_over_lock:
                weighted_average = 2
                lock_counter += 1
                if lock_counter >= 50:
                    take_over_lock = False
                    lock_counter = 0
        
            update_gui(behavior_label, accelerometer_label, detection_label, decision_label, root, behavior, accelerometer, detection, decision)
            update_gui(behavior_label, accelerometer_label, detection_label, weighted_avg_label, root2, behavior, accelerometer, detection, weighted_average)

        time.sleep(1 / 100)  # Adjust the delay as needed

    root.mainloop()  # Moved outside the loop

if __name__ == "__main__":
    main()


deque([0], maxlen=10)

Original decision: 0
deque([0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)

Original decision: 0

TclError: invalid command name ".!label"